# Load Training data

In [1]:
import pandas as pd 
from sklearn import preprocessing

#choose these features according to papers I read about the features that affect the intrusion attack 
df = pd.read_csv("traindata.csv")

X = df.drop(['Class','ID'],axis=1)
y = df.Class




## Encode the catecorigal data

In [2]:
from sklearn.preprocessing import LabelEncoder
for i in ['protocol_type','service','flag']:
  le = LabelEncoder()
  le.fit(X[i])
  X[i] = le.transform(X[i])


In [3]:
X.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,level
0,0,1,22,9,295,2584,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,3,0.0,0.0,0.0,0.0,1.0,0.0,0.67,101,255,1.00,0.00,0.01,0.03,0.0,0.0,0.00,0.01,21
1,0,1,22,9,312,839,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,7,0.0,0.0,0.0,0.0,1.0,0.0,0.29,223,255,1.00,0.00,0.00,0.02,0.0,0.0,0.01,0.01,21
2,0,1,18,9,1225,2451,0,0,0,28,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.00,255,116,0.45,0.02,0.00,0.00,0.0,0.0,0.00,0.00,11
3,0,1,22,9,293,296,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,4,0.0,0.0,0.0,0.0,1.0,0.0,0.50,216,255,1.00,0.00,0.00,0.02,0.0,0.0,0.00,0.02,21
4,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0.0,0.0,0.5,1.0,0.5,1.0,0.00,115,1,0.01,0.03,0.01,0.00,0.0,0.0,0.01,1.00,11


## Feature scaling

In [4]:
from sklearn.decomposition import PCA
from imblearn.under_sampling import NeighbourhoodCleaningRule
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import ADASYN 

scaler = preprocessing.RobustScaler().fit(X)   #scaling
names = X.columns
X_resample = scaler.transform(X)
X_resample = pd.DataFrame(X_resample, columns=names)



/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


## Over sampling

In [5]:
ada = ADASYN(random_state=100) #oversampling
X_resample, y = ada.fit_resample(X_resample, y)
X_resample = pd.DataFrame(X_resample, columns=names)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [6]:
X_resample

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,level
0,0.000000,0.0,0.000000,0.000000,0.913558,5.316872,0.0,0.0,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-0.090278,-0.312500,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.67,-0.916667,0.808163,0.568421,-0.428571,0.166667,3.0,0.000000,0.0,0.000000,0.010000,0.333333
1,0.000000,0.0,0.000000,0.000000,0.975432,1.726337,0.0,0.0,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-0.069444,-0.062500,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.29,-0.190476,0.808163,0.568421,-0.428571,0.000000,2.0,0.000000,0.0,0.010000,0.010000,0.333333
2,0.000000,0.0,-0.148148,0.000000,4.298453,5.043210,0.0,0.0,0.0,28.000000,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,-0.097222,-0.437500,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.240816,-0.010526,-0.142857,0.000000,0.0,0.000000,0.0,0.000000,0.000000,-3.000000
3,0.000000,0.0,0.000000,0.000000,0.906278,0.609053,0.0,0.0,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-0.083333,-0.250000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.50,-0.232143,0.808163,0.568421,-0.428571,0.000000,2.0,0.000000,0.0,0.000000,0.020000,0.333333
4,0.000000,0.0,-0.777778,-2.000000,-0.160146,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-0.090278,-0.437500,0.0,0.0,0.500000,1.000000,-0.549451,16.666667,0.00,-0.833333,-0.228571,-0.473684,0.000000,0.166667,0.0,0.000000,0.0,0.010000,1.000000,-3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23955,6.000000,0.0,1.238563,-1.037980,-0.160146,0.021655,0.0,0.0,0.0,0.000000,0.0,0.593132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-0.097222,-0.437500,0.0,0.0,0.593132,0.593132,0.000000,0.000000,0.00,-1.463672,-0.228571,-0.252478,13.857143,3.669093,0.0,0.000000,0.0,0.930832,0.593132,-3.825184
23956,9871.039048,0.0,0.516673,-0.975552,0.657797,0.994336,0.0,0.0,0.0,0.878233,0.0,0.219558,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.219558,-0.091802,-0.388722,0.0,0.0,0.780442,0.780442,0.000000,0.000000,0.00,0.000000,-0.212840,-0.462129,7.884545,13.007364,0.0,0.035129,0.0,0.780442,0.780442,-2.471713
23957,9809.687523,0.0,0.703704,-1.250000,-0.156506,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-0.090278,-0.375000,0.0,0.0,1.000000,1.000000,0.000000,0.000000,0.00,0.000000,-0.224490,-0.473684,10.142857,16.666667,0.0,0.000000,0.0,1.000000,1.000000,-1.666667
23958,9952.343820,0.0,0.299558,-0.656960,1.603080,2.148611,0.0,0.0,0.0,1.897728,0.0,0.474432,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.474432,-0.093572,-0.404652,0.0,0.0,0.525568,0.525568,0.000000,0.000000,0.00,0.000000,-0.199316,-0.448714,5.262985,8.759466,0.0,0.075909,0.0,0.525568,0.525568,-3.406251


Here I use **RandomizedSearchCV** to hypertuning the parameters after I get the best parameters from the RandomizedSearchCV I use them and tune them again manual

In [7]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV


# xgb_model =xgb.XGBClassifier(objective= 'binary:logistic')
# params =[ {
#     "n_estimators":[5,10,15,20,30,40], # default 100
#     "booster":['gbtree','gblinear','dart'], # default gbtree
#     "eta": [0.1,0.01,0.001], # default 0.1 
#     "alpha": [5,8,10,15],
#     "num_parallel_tree":[3,6,9],
#     "colsample_bytree": [0.7,0.6,0.5,0.4,0.3],
#     "scale_pos_weight":[35,45,55,65,75,85],
#     "subsample":[0.3,0.5,0.8,1],
#     "max_depth": [5,10,20,30,40], # default 3
#     "nthread": [2,3,4,5],
#     "seed":[20,32,42,52,62]
# }]

# search = RandomizedSearchCV(xgb_model, n_iter=300,params, cv=3, verbose=1, n_jobs=1, return_train_score=True)
# search.fit(X_resample,y)

# print(search.best_params_)

xgb_model =xgb.XGBClassifier(objective= 'binary:logistic',n_estimators = 30,booster = 'gbtree',eta = 0.01,alpha = 8,num_parallel_tree = 6, colsample_bytree = 0.5,
                scale_pos_weight = 65,subsample = 1,n_jobs=-1, max_depth=5, nthread=4,seed=6002)

xgb_model.fit(X_resample,y)



XGBClassifier(alpha=8, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, eta=0.01, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=30, n_jobs=-1,
              nthread=4, num_parallel_tree=6, objective='binary:logistic',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=65,
              seed=6002, silent=None, subsample=1, verbosity=1)

# Load Test data

In [8]:
test = pd.read_csv('testdata.csv')
test = test.drop(['ID'],axis=1)

for i in ['protocol_type','service','flag']:
  le = LabelEncoder()
  le.fit(test[i])
  test[i] = le.transform(test[i])


newtest=scaler.transform(test)
newtest = pd.DataFrame(newtest, columns=names)

y_pred = xgb_model.predict(newtest)


In [9]:
submission = pd.DataFrame({'ID':range(len(y_pred)),'Class': y_pred})
submission.to_csv('submission59.csv', index=False)